In [ ]:
!pip install binarytree==6.5.1
# restart runtime

# https://pypi.org/project/binarytree/

     |████████████████████████████████| 1.1 MB 19.4 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import os
import random
import numpy as np 
import pandas as pd
from binarytree import Node, build

In [ ]:
class Solution():
    """Class that represents a condidate solution to be evolved"""

    def __init__(self,X):
        self.data = X
        self.features = X.columns
        self.score = -1
        self.metrics = {}
        #self.operators = ['<', '<=', '==', '>', '>=']
        self.operators = ['<=', '>=']


    def generate_rule(self):
        features = self.features
        feature = random.choice(features)
        operators = self.operators
        operator = random.choice(operators)
        thresholds = self.data[feature].unique()
        threshold = random.choice(thresholds)
        return f"d['{feature}'] {operator} {threshold}"


    def generate_tree(self, height, value='root'):
        stop=0
        root=0
        if value=='root':
            root=1
            value = random.choice(['and','or'])
            node = Node(value)
            value = random.choice(['and','or',self.generate_rule()])
        else:
            value = random.choice(['and','or',self.generate_rule()])
            if height==0:
                value = self.generate_rule()
            node = Node(value)
            
        if len(value)>3 and root==0: # parent node is a rule and not root
            stop=1
        
        if height>0 and not stop:
            node.left = self.generate_tree(height-1,value)
        if height>0 and not stop:
            node.right = self.generate_tree(height-1,value)

        self.tree = node
        return node


    def load_tree(self, values):
        self.tree = build(values)


    def create_random(self, max_depth=3):
        height = random.choice(range(2,max_depth))
        binary_tree = self.generate_tree(height)
        self.tree = binary_tree
            

    def print_solution(self):
        print(self.tree)
        

In [ ]:
class GeneticEvolution():
    """Class that holds a genetic algorithm for evolving a population of condidate solutions"""

    def __init__(self, X, y, random_select=0.1, mutate_chance=0.1, retain=0.3):
        self.X = X
        self.y = y
        self.random_select = random_select
        self.mutate_chance = mutate_chance
        self.retain = retain

    
    def create_population(self, pop_count, max_depth=3):
        """Create a population of random solutions"""
        population = []
        for _ in range(pop_count):
            # Create a random solution
            solution = Solution(self.X)
            solution.create_random(max_depth)
            # Add the solution to the population
            population.append(solution)
        return population

    
    def print_population(self, population):
        """Display a binary tree representation of a given solution"""
        for solution in population:
            solution.print_solution()


    def fitness(self,solution):
        """Return the fitness of a solution and calculates it metrics"""

        map = {
            0 : False, # pass
            1 : True   # fail
        }
        TP, TN, FP, FN = 0, 0, 0, 0
        
        for i in range(self.X.shape[0]):
            d = self.X.iloc[i]
            rule = str(solution.tree.inorder).replace('Node(','').replace('),','')[1:-2]
            detected = eval(rule)
            truth = map[self.y.iloc[i]]
            if detected==truth:
                if truth==True:
                    TP+=1
                if truth==False:
                    TN+=1
            if detected!=truth:
                if truth==True:
                    FP+=1
                if truth==False:
                    FN+=1
        TPR = TP/(TP+FN) if TP or FN else 0
        FPR = FP/(FP+TN) if FP or TN else 0 # TP and TN can be ==0 if all rules are evaluated positive
        
        fitness = np.sqrt((1-TPR)**2+FPR**2)/np.sqrt(2)
        solution.score=fitness

        # Calculating metrics
        recall = TPR
        precision = TP/(TP+FP) if TP or FP else 0
        solution.metrics['recall'] = recall
        solution.metrics['precision'] = precision
        solution.metrics['f1score'] = 2*precision*recall/(precision+recall) if precision or recall else 0
        solution.metrics['accuracy'] = (TP+TN)/(TP+TN+FP+FN)
        solution.metrics['AUC'] = (1+TPR-FPR)/2

        return fitness


    def grade_population(self, population):
        """Return the average fitness of a population"""
        res = 0
        for solution in population:
            res += self.fitness(solution)
        return res / float((len(population)))


    def mutate(self, solution):
        """Randomly mutate one part of the solution/ one node of the tree"""
        # Choose a random Node
        tree_values = solution.tree.values
        possible_mutation_indices = list(range(0,len(tree_values)-1))
        mutation_index = random.choice(possible_mutation_indices)
        while tree_values[mutation_index]==None:
            possible_mutation_indices.remove(mutation_index)
            mutation_index = random.choice(possible_mutation_indices)
        # Mutate the node
        if len(tree_values[mutation_index])<4: # and/or
            tree_values[mutation_index] = random.choice(['and','or'])
        else:                                  # expr
            tree_values[mutation_index] = random.choice([solution.generate_rule()])
        mutated_sol = Solution(self.X)
        mutated_sol.load_tree(tree_values)
        return mutated_sol


    def crossover(self, parent1, parent2):
        """Make two children as parts of their parents"""
        child1, child2 = Solution(self.X), Solution(self.X)
        child1.load_tree(parent1.tree.values)
        child2.load_tree(parent2.tree.values)

        # Randomly switch left/right subtree
        # modify this later
        r = random.randint(1,4)
        if r==1:
            child1.tree.left = parent2.tree.left
            child2.tree.left = parent1.tree.left
        if r==2:
            child1.tree.left = parent2.tree.right
            child2.tree.right = parent1.tree.left
        if r==3:
            child1.tree.right = parent2.tree.left
            child2.tree.left = parent1.tree.right
        if r==4:
            child1.tree.right = parent2.tree.right
            child2.tree.right = parent1.tree.right

        # Randomly mutate some of the children
        if random.random() > self.mutate_chance:
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
        return child1, child2
    

    def evolve(self, population):
        """Evolve a population of solutions by one generation"""

        # Calculate the fitness of each individual in the population
        graded_soutions = [(self.fitness(solution), solution) for solution in population]

        # Sort the population according to the fitness
        graded_soutions.sort(key=lambda t:t[0]) # ascending, min is better
        graded_soutions = [sol[1] for sol in graded_soutions]

        # Move a portion of the population to the next generation without mutations
        retain_length = int(len(graded_soutions)*self.retain)
        retain_length = max(retain_length, 2) if retain_length>=2 else 1
        new_population = graded_soutions[:retain_length]

        # For the individuals we aren't keeping, randomly keep some anyway
        for individual in graded_soutions[retain_length:]:
            if self.random_select > random.random():
                new_population.append(individual)
                
        # Now find out how many spots we have left to fill
        new_pop_length = len(new_population)
        desired_length = len(population) - new_pop_length

        # Add children, which are bred from two random solutions
        parents = new_population
        if new_pop_length > 1 and desired_length> 0:
            children = []
            while len(children) < desired_length:
                if new_pop_length==2:
                    male_index = 1
                    female_index = 0
                else:
                    male_index = random.randint(0, new_pop_length-1)
                    female_index = random.randint(0, new_pop_length-1)
                
                # Assuming they aren't the same solutions
                if male_index != female_index:
                    male = parents[male_index]
                    female = parents[female_index]
                    # crossover them.
                    babies = self.crossover(male, female)
                    # Add the children one at a time
                    for baby in babies:
                        # keep the population size under the desired length
                        if len(children) < desired_length:
                            children.append(baby)
            parents.extend(children)
            
        return parents


In [ ]:
def Evolution(X, y, nbr_generation, population_size, max_depth, random_select, mutate_chance, retain, verbose=1):
    """Uses a Genetic Algorithm to find the best detection rule"""

    ga = GeneticEvolution(X, y, random_select, mutate_chance, retain)

    i = 0
    population = ga.create_population(population_size, max_depth)
    if verbose:
        avg_fit = ga.grade_population(population)
        print(f"Generation {i} | Average fitness {round(avg_fit,2)}")

    for i in range(1, nbr_generation):
        new_population = ga.evolve(population)
        if verbose:
            avg_fit = ga.grade_population(new_population) 
            print(f"Generation {i} | Average fitness {round(avg_fit,2)}")
        population = new_population

    graded_soutions = [(ga.fitness(solution), solution) for solution in population]
    graded_soutions.sort(key=lambda t:t[0]) 

    best = graded_soutions[0][1]

    return best


In [ ]:
def evaluate_solution(X, y, solution):
    """Evaluates a given solution"""
    
    map = {
        0 : False, # pass
        1 : True   # fail
    }

    TP, TN, FP, FN = 0, 0, 0, 0

    for i in range(X.shape[0]):
        d = X.iloc[i]
        rule = str(solution.tree.inorder).replace('Node(','').replace('),','')[1:-2]
        detected = eval(rule)
        truth = map[y.iloc[i]]
        if detected==truth:
            if truth==True:
                TP+=1
            if truth==False:
                TN+=1
        if detected!=truth:
            if truth==True:
                FP+=1
            if truth==False:
                FN+=1
    TPR = TP/(TP+FN) if TP or FN else 0
    FPR = FP/(FP+TN) if FP or TN else 0 # TP and TN can be ==0 if all rules are evaluated positive

    # Calculating metrics
    recall = TPR
    precision = TP/(TP+FP) if TP or FP else 0
    eval_metrics = {}
    eval_metrics['recall'] = recall
    eval_metrics['precision'] = precision
    eval_metrics['f1score'] = 2*precision*recall/(precision+recall) if precision or recall else 0
    eval_metrics['accuracy'] = (TP+TN)/(TP+TN+FP+FN)
    eval_metrics['AUC'] = (1+TPR-FPR)/2
    
    # Confusion Matrix
    confusion = (np.array([TP,FP,FN,TN]).reshape(2,2))
    print(confusion)

    return eval_metrics

In [ ]:
dataset_path = '/mnt/d/PFE/Code/dataset'

from google.colab import drive
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/CI/replicating-ets/DL-CIBuild/dataset'

Mounted at /content/drive


In [ ]:
# Loading the dataset

df=pd.DataFrame(
    columns = ['build_Failed', 'gh_is_pr', 'git_prev_commit_resolution_status',
       'gh_team_size', 'gh_num_commit_comments', 'git_diff_src_churn',
       'git_diff_test_churn', 'gh_diff_files_added', 'gh_diff_files_deleted',
       'gh_diff_files_modified', 'gh_diff_tests_added',
       'gh_diff_tests_deleted', 'gh_diff_src_files', 'gh_diff_doc_files',
       'gh_diff_other_files', 'gh_sloc', 'gh_test_lines_per_kloc',
       'gh_test_cases_per_kloc', 'gh_asserts_cases_per_kloc', 'tr_build_id',
       'gh_build_started_at'],
    dtype='object')


for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        if filename[-4:]==".csv":
            df = pd.concat([df, pd.read_csv(os.path.join(dirname, filename))])
            

In [ ]:
X = df.iloc[:,1:19]
y = df.iloc[:,0] # 'build_failed'

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
nbr_generation = 10
population_size = 10
max_depth = 5
random_select = 0.2
mutate_chance = 0.2
retain = 0.3
verbose = 0

solution = Evolution(
    X_train, y_train,
    nbr_generation, population_size,
    max_depth, random_select, mutate_chance, retain, verbose
)

print("Found solution:")
solution.print_solution()

print("Solution's metrics on the training data:")
print(solution.metrics)

print("\nSolution's metrics on the validation data:")
print(evaluate_solution(X_val, y_val, solution))

Found solution:

                                      ___________________________________________________________________and_____________________
                                     /                                                                                           \
                  _________________and__________________________________                                     d['git_prev_commit_resolution_status'] <= 0
                 /                                                      \
d['gh_diff_files_deleted'] <= 3187                      ________________and_____________
                                                       /                                \
                                       d['gh_diff_files_deleted'] <= 28     d['gh_diff_doc_files'] <= 87

Solution's metrics on the training data:
{'recall': 1.0, 'precision': 0.00021646895835137242, 'f1score': 0.00043284421936545033, 'accuracy': 0.6839346326508267, 'AUC': 0.8419565009102232}

Solution's metrics on

Debugging examples

In [ ]:
# crossover
ga = GeneticEvolution(df)
pop=ga.create_population(2)
pop[0].print_solution()
pop[1].print_solution()
c1,c2=ga.crossover(pop[0],pop[1])
c1.print_solution()
c2.print_solution()


                                    ____________________________________and_____________________________________________
                                   /                                                                                    \
                 ________________and________________                                               _____________________and__________
                /                                   \                                             /                                  \
d['gh_diff_tests_deleted'] <= 92     d['gh_diff_files_modified'] >= 106     d['git_prev_commit_resolution_status'] >= 0     d['gh_sloc'] >= 189987


                     _________________________________or_______________________________
                    /                                                                  \
          _________or_______________                                     _______________or_____________
         /                          \                    

In [ ]:
# mutation
ga = GeneticEvolution(df)
pop=ga.create_population(2)
pop[1].print_solution()
new= ga.mutate(pop[1])
new.print_solution()


                                    ______________________________or___________________
                                   /                                                   \
                  ________________or_____________                              _________or___________
                 /                               \                            /                      \
d['git_diff_test_churn'] >= 14814    d['gh_diff_doc_files'] >= 39    d['gh_is_pr'] >= 0    d['gh_team_size'] <= 219


                                              ______________________________or___________________
                                             /                                                   \
                       _____________________or_____________                              _________or___________
                      /                                    \                            /                      \
d['git_prev_commit_resolution_status'] >= 2    d['gh_diff_doc_files'] >= 39  

In [ ]:
# metrics/fitness
ga = GeneticEvolution(df)
pop=ga.create_population(5)
pop[4].print_solution()
ga.fitness(pop[4])
pop[4].metrics


                            __________________________________or___________________________________
                           /                                                                       \
             ____________and_______________                                        ________________and________________
            /                              \                                      /                                   \
d['gh_team_size'] >= 101     d['gh_diff_files_added'] <= 2910    d['gh_num_commit_comments'] <= 36     d['gh_num_commit_comments'] >= 37



{'recall': 0.2455218617771509,
 'precision': 0.4802897051215727,
 'f1score': 0.32493729218923173,
 'accuracy': 0.4050637335467667,
 'AUC': 0.36644038103580423}